Copyright 2019 Carsten Blank

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
%aimport lib_experimental_utils
%aimport lib_experiment_setups

In [4]:
import numpy as np
import lib_experimental_utils as lib
from lib_experimental_utils import FinishedExperiment, save
import lib_experiment_setups as expset

In [64]:
readout_swap = {}
id1 = expset.create_regular_experiment_and_then_simulation(backend_enum=expset.BackendEnum.IBMQ_OURENSE, 
                                                           instead_general_weights_use_hadamard=False,
                                                           use_barriers=False, readout_swap=readout_swap,
                                                           no_experiment=True, dont_use_dask=False)

In [48]:
expset.get_ids()

In [66]:
loaded_data = expset.load_by_index(0, 'exp_sim_regular_')
loaded_data

In [67]:
experiment: FinishedExperiment = loaded_data[0]
simulation: FinishedExperiment = loaded_data[1]
experiment.backend_name

In [68]:
w_1 = 0.5
w_2 = 1 - w_1
theta = np.asarray(experiment.theta)
theory_classification = w_1 * np.sin(theta/2 + np.pi/4)**2 - w_2 * np.cos(theta/2 + np.pi/4)**2

experiment.show_plot(compare_classification=theory_classification, classification_label='experiment', compare_classification_label='theory')
simulation.show_plot(compare_classification=theory_classification, classification_label='simulation (noise)', compare_classification_label='theory')
experiment.show_plot(compare_classification=simulation.get_classification(), classification_label='experiment', compare_classification_label='simulation')

In [69]:
from scipy.optimize import minimize

def theory_expectation(w_1, w_2):
    def inner(x):
        return w_1 * np.sin(x/2 + np.pi/4)**2 - w_2 * np.cos(x/2 + np.pi/4)**2
    return inner


def mse(classification, theta):
    classification = np.asarray(classification)
    def inner(x):
        a, vartheta, w_1 = x
        reference = np.asarray([
            a*theory_expectation(w_1=w_1, w_2=1 - w_1)(t - vartheta) for t in theta
        ])
        return np.sqrt(sum(np.power(classification - reference, 2)))
    return inner

fun = mse(experiment.get_classification(), theta)
x_0 = [1.0, 0, 0]
result = minimize(fun, x_0)

from sympy import nsimplify

[a, vartheta, w_1] = result.x
"amplitude dampening: {:.4}, shift: {} pi, approx. w_1: {:.4}".format(
    a, 
    nsimplify(vartheta/np.pi, tolerance=0.1), 
    w_1)

In [18]:
lib.save(directory="../experiment_results", experiment=experiment, simulation=simulation)

In [43]:
#simulation.parameters['device_properties']
